# Rekonstruktion einer Baumstruktur mit pc-skeletor
Dieses Notebook verwendet das GitHub-Projekt [pc-skeletor](https://github.com/meyerls/pc-skeletor), um aus einer 3D-Punktwolke das topologische Skelett zu extrahieren. Der Algorithmus basiert auf einer Voxelisierung, Distanztransformation und Graphanalyse.

## Voraussetzungen
- Python ≥ 3.6
- GitHub-Repo: https://github.com/meyerls/pc-skeletor
- Installation:
```bash
git clone https://github.com/meyerls/pc-skeletor.git
cd pc-skeletor
pip install -e .
```

In [3]:
import open3d as o3d
import numpy as np
import pandas as pd
from pc_skeletor import Dataset, LBC

# === Schritt 1: Beispiel-Datensatz herunterladen ===
downloader = Dataset()
trunk_pcd_path, branch_pcd_path = downloader.download_semantic_tree_dataset()

pcd_trunk = o3d.io.read_point_cloud(trunk_pcd_path)
pcd_branch = o3d.io.read_point_cloud(branch_pcd_path)
pcd = pcd_trunk + pcd_branch
print("Punktwolke geladen (Trunk + Branches)")

# === Schritt 2: Laplacian-Based Contraction (LBC) ===
lbc = LBC(point_cloud=pcd, down_sample=0.008)
lbc.extract_skeleton()
lbc.extract_topology()
print("Skelettierung abgeschlossen")

# === Schritt 3: Kantenlängen berechnen ===
edges = np.array(lbc.skeleton_graph.edges)
vertices = np.asarray(lbc.skeleton.points)

lengths = []
for start_idx, end_idx in edges:
    start_point = vertices[start_idx]
    end_point = vertices[end_idx]
    length = np.linalg.norm(start_point - end_point)
    lengths.append(length)

lengths = np.array(lengths)
total_length = np.sum(lengths)

print(f"Anzahl Äste (Kanten): {len(lengths)}")
print(f"Gesamtlänge aller Äste: {total_length:.4f} m")
print(f"Durchschnittliche Astlänge: {np.mean(lengths):.4f} m")

# === Schritt 4: Ergebnisse als CSV speichern ===
df = pd.DataFrame({
    'Edge_Start': edges[:, 0],
    'Edge_End': edges[:, 1],
    'Length_m': lengths
})
output_csv = "skeleton_edge_lengths.csv"
df.to_csv(output_csv, index=False)
print(f"Ergebnisse gespeichert in: {output_csv}")

# === Schritt 5: Visualisierung und Export ===
lbc.visualize()
lbc.export_results('./output')
print("Export abgeschlossen: ./output")


ModuleNotFoundError: No module named 'open3d'

## 1. Punktwolke einlesen (echter Baum)

In [ ]:
import numpy as np
import pandas as pd

# Punktwolke als CSV (x, y, z-Spalten)
data = pd.read_csv("your_tree_pointcloud.csv")  # <- Datei anpassen
points = data[['x', 'y', 'z']].to_numpy()
print("Punktanzahl:", points.shape)


## 2. Skeletonisierung mit pc-skeletor

In [ ]:
from skeletor import Skeletonizer
from skeletor.voxelize import voxelize

# Voxelgröße (abhängig von Baumgröße anpassen)
voxel_size = 0.05

# Voxelisiere Punktwolke
volume, origin, spacing = voxelize(points, voxel_size=voxel_size)

# Skeletonizer initialisieren und ausführen
skel = Skeletonizer(volume)
skel.skeletonize(method='zhang')  # andere Methoden: 'medial', 'thinning'
graph = skel.to_graph(min_branch_length=2)

print(f"Anzahl Knoten im Skelett: {graph.number_of_nodes()}")


## 3. Visualisierung des extrahierten Skeletts

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')

# Originalpunkte (optional)
ax.scatter(points[:,0], points[:,1], points[:,2], s=1, alpha=0.05)

# Skelettpunkte plotten
for u, v in graph.edges():
    p1 = np.array(graph.nodes[u]['coord']) * spacing + origin
    p2 = np.array(graph.nodes[v]['coord']) * spacing + origin
    ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], 'r-', linewidth=1)

ax.set_title("Topologisches Skelett des Baums (pc-skeletor)")
plt.show()
